# Track features in scRNA-seq

In [ ]:
!lamin init --storage ./test-scrna --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

ln.settings.verbosity = 3  # show hints

## Track

We're working with mouse data, so let's set a context:

In [ ]:
lb.settings.species = "mouse"

Let's look at a scRNA-seq count matrix in form of an `AnnData` object:

In [ ]:
adata = ln.dev.datasets.anndata_mouse_sc_lymph_node()

In [ ]:
adata

The feature identifiers of this matrix are stored in a `DataFrame`:

In [ ]:
adata.var.head()

When we create a file object from an AnnData, we'll automatically link it's FeatureSet:

In [ ]:
file = ln.File.from_anndata(
    adata, description="Detmar22", var_ref=lb.Gene.ensembl_gene_id
)

In [ ]:
file.save()

The file now has two feature sets linked:

In [ ]:
feature_sets = file.feature_sets.all()

for feature_set in feature_sets:
    print(feature_set)

In [ ]:
ln.Feature.select(feature_sets__in=feature_sets).df().head()

In [ ]:
lb.Gene.select(feature_sets__in=feature_sets).df().head(2)

:::{dropdown} Annotate an existing file

You can annotate an existing file via:

```
feature_set = ln.FeatureSet.from_values(adata.var.index, lb.Gene.ensembl_gene_id)
feature_set.save()
file.feature_sets.add(feature_set)
file.save()
```

:::

## Query by features

Lookup one the genes in our Gene registry.

In [ ]:
genes = lb.Gene.lookup()

Query all feature sets that contain the gene:

In [ ]:
ln.FeatureSet.select(genes=genes.actg1).df()

Query files whose features contain the gene:

In [ ]:
ln.File.select(feature_sets__genes=genes.actg1).df()

## Add more datasets

Let's consider another dataset with less curated features:

In [ ]:
lb.settings.species = "human"

In [ ]:
pbcm68k = ln.dev.datasets.anndata_pbmc68k_reduced()

We see that this dataset is indexed by gene symbols: 

In [ ]:
pbcm68k.var.index

In [ ]:
ln.File.from_anndata(
    pbcm68k, description="10x reference pbmc68k", var_ref=lb.Gene.symbol
).save()

In [ ]:
conde22 = ln.dev.datasets.anndata_human_immune_cells()

In [ ]:
conde22.var.index

In [ ]:
ln.File.from_anndata(
    conde22, description="Conde22", var_ref=lb.Gene.ensembl_gene_id
).save()

In [ ]:
ln.File.select().df()

In [ ]:
ln.FeatureSet.select().df()

## Query across datasets

In [ ]:
feature_set = ln.FeatureSet.select(id="pGRHEkO_6ohpysDpdzaZ").one()

In [ ]:
gene_set = feature_set.genes.all()

In [ ]:
ln.FeatureSet.select().df()

In [ ]:
from django.db.models import Count, Q

In [ ]:
len_gene_set = len(gene_set)

ln.FeatureSet.select().annotate(
    n_genes=Count("genes"), n_match=Count("genes", filter=Q(genes__in=gene_set))
).filter(n_genes=len_gene_set, n_match=len_gene_set).df()

In [ ]:
!lamin delete test-scrna
!rm -r ./test-scrna